In [1]:
import re
import pandas as pd
import requests 
import json

### Starting of classification Tag evaluation API

In [16]:
merged_corpus = pd.read_csv('/home/ifte/resource/docomo_dic/data/merged_corpus.csv')

In [18]:
merged_corpus.iloc[519].title

'BMWのレンタカー利用時に事故や車両の故障・不具合が発生した場合はどこに連絡すればいいですか？ | よくあるご質問| カーシェアならdカーシェア | ドコモのカーシェアリングサービス'

In [ ]:
merged_corpus.iloc[519].title

In [2]:
raw_corpus = pd.read_csv('/home/ifte/resource/docomo_dic/data/corpus.csv')

In [10]:
url_list = []
ques = []
ind = 1
collect = []
for index, col in test_df.iterrows():
    url_list.append(col['url'])
    ques.append(col['question'])
    page_id = list(set(corpus_df[(corpus_df['url'] == col['url'])].page.values))
    # print(col['url'], page_id, col['question'], col['tag'])
    if page_id:
        collect.append(page_id[0])
    else:
        collect.append(99999)
    ind += 1

### Converter DocID => URLs

In [12]:
# defining the api-endpoint  
API_ENDPOINT = "http://127.0.0.1:5000/api/v1.0/predict/"
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
result = []
data = {
"output_dir": "/home/ifte/resource/docomo_dic/model", 
"agent_id": -1, 
"query": "BMWのレンタカー利用時に事故や車両の故障・不具合が発生した場合はどこに連絡すればいいですか？",
"classification": "ｄカーシェア",
"max_answer": 5, 
"data_dir": "/home/ifte/resource/docomo_dic/data", 
"keyphrase_extraction": 0
}

r = requests.post(url = API_ENDPOINT, data=json.dumps(data), headers = headers) 

# json.loads(r.text)

    

In [19]:
list(set(raw_corpus[raw_corpus['page'] == 519].url.values))[0]


'https://faq.carshare.dmkt-sp.jp/faq/show/402'

In [13]:
for i in json.loads(r.text)['page_relevance']:
    print(list(set(raw_corpus[raw_corpus['page'] == i['doc_id']].url.values))[0], i['doc_id'])


https://faq.carshare.dmkt-sp.jp/faq/show/349 501
https://faq.carshare.dmkt-sp.jp/faq/show/361 326
https://faq.carshare.dmkt-sp.jp/faq/show/395 286
https://faq.carshare.dmkt-sp.jp/faq/show/394 289
https://faq.carshare.dmkt-sp.jp/faq/show/389 319


In [19]:
json.loads(r.text)

{'page_relevance': [{'doc_id': 242,
   'score': 78.9449886536596,
   'keywords': ['dカーシェア',
    'カリ',
    'テコ',
    '登録',
    'た',
    '電子',
    '鍵',
    'smartphone',
    'です'],
   'snippet': ' カリテコのクルマの電子鍵の登録方法を教えてください。 No : 240 更新日時 : 2020/07/15 16:41 以下の方法で一度登録が完了いたしますと、以後のご利用に電子鍵としてご利用いただ...'},
  {'doc_id': 512,
   'score': 50.53255968795844,
   'keywords': ['dカーシェア', 'カリ', 'テコ', 'いい', 'です'],
   'snippet': ' カリテコで予約時にエラーが出てしまいます。どのようにすればいいでしょうか？ No : 165 更新日時 : 2020/07/15 16:36 エラー番号をお控えの上、dカーシェアサポートセンター に...'},
  {'doc_id': 160,
   'score': 49.55697658195059,
   'keywords': ['dカーシェア', '登録', 'た', '電子', '鍵', 'smartphone', 'です'],
   'snippet': ' オリックスのクルマの電子鍵の登録方法を教えてください。 No : 238 更新日時 : 2020/07/15 16:41 以下の方法で一度登録が完了いたしますと、以後のご利用に電子鍵としてご利用いた...'},
  {'doc_id': 159,
   'score': 48.17624693805779,
   'keywords': ['dカーシェア', 'カリ', 'テコ', '登録', 'た', '電子', '鍵', 'です'],
   'snippet': '...い合わせ先 dカーシェアサポートセン...】を選択： カリテコ・センタ...選択： カリテコ・センター(...アの電子鍵の登録に関しまして...せ内容に応じた番号を選択し...カーシェアの電子鍵の登録に関.

In [61]:
# defining the api-endpoint  
API_ENDPOINT = "http://127.0.0.1:5000/api/v1.0/predict/"
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
result = []
for index, col in test_df.iterrows():
    # data to be sent to api 
    data = {'output_dir':'/home/iftekhar/amiebot/Resources/876/model', 
            'data_dir':'/home/iftekhar/amiebot/Resources/876/data', 
            'agent_id':-1, 
            'query':col['question'], 
            'max_answer':5,
            'classification': col['tag']
           } 
    
    r = requests.post(url = API_ENDPOINT, data=json.dumps(data), headers = headers) 
    result.append(r.text)
    

In [62]:
len(result)

154

In [ ]:
result

In [203]:
r = json.dumps(result)

In [204]:
file = open(path.join('/home/iftekhar/Downloads/', "output.json"), "w", encoding='utf-8')
json.dump(r, file, ensure_ascii=False)
file.close()
    

In [ ]:
j = 0
s = 0
for items in saver.values():
    #print(items)
    if items:
        z = [x for x in items.keys()][1:]
        score = mean_reciprocal_rank_score(items['actual'], z)
        if not score > 0:
            s += 1
            print("Not Found: ", ques[j], "\nCorrect", url_list[j])
            for ii in z:
                print('Predict:', ' '.join(list(set(corpus_df[(corpus_df['page'] == ii)].url.values))))
        print(j,':', items['actual'], '->', z, '->', score)
    j += 1
print('Failed ', s )

In [10]:
def mean_reciprocal_rank_score(actual_value, predicted_values):
    if not actual_value:
        return 0
    pos = 0
    val = 0
    for i in predicted_values:
        if i == actual_value and pos == 0:
            val = 1
            break
        elif i == actual_value and pos == 1:
            val = 0.5
            break
        elif i == actual_value and pos == 2:
            val = 0.33
            break
        else:
            val = 0
        pos += 1

    return val

def cut_string(text):
    try:
        # print(text[:text.index("?")])
        return text[:text.index("?")]
    except ValueError:
        # print(text)
        return text

In [8]:
# test_data = '/home/iftekhar/Documents/test_data_docomo_07.12.2020_red.csv'
# test_data = '/home/iftekhar/Documents/test_data_docomo_07.12.2020.csv'
# test_data = '/home/ifte/Documents/test_data_docomo_except.csv'
# test_data = '/home/ifte/Documents/test_data_few_exceptional.csv'
test_data = '/home/ifte/Downloads/classification_tag_10.12.2020_part2.csv'
test_df = pd.read_csv(test_data)

In [9]:
test_df.rename(columns={'correct_url': 'url'}, inplace=True)

In [ ]:
test_df.url.values

In [11]:
test_df.url = test_df.url.apply(lambda x: cut_string(x))

In [12]:
# defining the api-endpoint  
API_ENDPOINT = "http://127.0.0.1:5000/api/v1.0/predict/"
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
result = []
for index, col in test_df.iterrows():
    # data to be sent to api 
    data = {'output_dir':'/home/ifte/resource/docomo_dic/model', 
            'data_dir':'/home/ifte/resource/docomo_dic/data', 
            'agent_id':-1, 
            'query':col['question'],
            'classification': col['tag'],
            'max_answer':5
           } 
    
    r = requests.post(url = API_ENDPOINT, data=json.dumps(data), headers = headers) 
    result.append(r.text)
    

In [13]:
from collections import defaultdict
saver = defaultdict(list)

j = 0
for items in result:
    for i in json.loads(items)['page_relevance']:
        saver[j].append(list(set(raw_corpus[raw_corpus['page'] == i['doc_id']].url.values))[0])
        saver[str(str(j)+'doc_id')].append(i['doc_id'])
    j += 1
        

In [14]:
collector = defaultdict(list)
i = 0

for index, col in test_df.iterrows():
    collector['question'].append(col['question'])
    collector['tag'].append(col['tag'])
    collector['correct_url'].append(col['url'])
    collector_correct = []
    try:
        collector_correct = list(set(raw_corpus[raw_corpus['url'] == col['url']].page.values))[0]
    except IndexError:
        continue
        pass
    collector['correct_id'].append(collector_correct)
    collector['returned_id'].append(saver[str(str(i)+'doc_id')])
    score = mean_reciprocal_rank_score(collector_correct, saver[str(str(i)+'doc_id')])
    if score:
        collector['mrr_score'].append(score)
    else:
        try:
            url_loc = saver[i].index(col['url'])
            if url_loc == 1:
                collector['mrr_score'].append(0.5)
            elif url_loc == 2:
                collector['mrr_score'].append(0.33)
            elif url_loc == 0:
                collector['mrr_score'].append(1)
            elif url_loc >= 3:
                collector['mrr_score'].append('LOW_RANK')
    
        except ValueError:
            collector['mrr_score'].append('NOT_MATCHED')
            pass

    collector['urls'].append(saver[i])
#         print('ok ', i)
    i += 1

    

In [15]:
new_df = pd.DataFrame.from_dict(collector, orient='index').T

In [16]:
new_df.head(10)

,question,tag,correct_url,correct_id,returned_id,mrr_score,urls
0,dポイントとは何ですか？,ｄマーケット,https://d.dmkt-sp.jp/common/qa/category/dpoint...,137,"[137, 3, 140, 95, 104]",1,[https://d.dmkt-sp.jp/common/qa/category/dpoin...
1,ポイントが利用できません。,ｄトラベル,https://travel.dmkt-sp.jp/faq/14/,769,"[816, 938, 666, 677, 1272]",NOT_MATCHED,"[https://travel.dmkt-sp.jp/faq/87/, https://tr..."
2,ポイントについて教えてください。,ｄトラベル,https://travel.dmkt-sp.jp/faq/15/,942,"[801, 1009, 962, 940, 783]",NOT_MATCHED,"[https://travel.dmkt-sp.jp/faq/56/, https://tr..."
3,予約した後にクーポンを利用できますか。,ｄトラベル,https://travel.dmkt-sp.jp/faq/25/,923,"[712, 993, 706, 705, 710]",NOT_MATCHED,"[https://travel.dmkt-sp.jp/coupon/20201118/, h..."
4,何日先の予約まで可能ですか。,ｄトラベル,https://travel.dmkt-sp.jp/faq/73/,1022,"[1278, 815, 1252, 674, 619]",NOT_MATCHED,"[nan, https://travel.dmkt-sp.jp/faq/151/, nan,..."
5,何枚まで購入できますか。,ｄトラベル,https://travel.dmkt-sp.jp/faq/119/,1037,"[677, 156, 90, 783, 997]",NOT_MATCHED,"[https://travel.dmkt-sp.jp/howto/lecoupon/, ht..."
6,dカーシェアで利用できるレンタカーサービスは何がありますか？,ｄカーシェア,https://faq.carshare.dmkt-sp.jp/faq/show/4,536,"[536, 323, 470, 221, 468]",1,"[https://faq.carshare.dmkt-sp.jp/faq/show/4, h..."
7,利用規約に違反した場合、どの様な対応が取られますか？,ｄカーシェア,https://faq.carshare.dmkt-sp.jp/faq/show/9,551,"[551, 228, 1057, 221, 1054]",1,"[https://faq.carshare.dmkt-sp.jp/faq/show/9, h..."
8,マイカーシェアとは、どんなサービスですか？,ｄカーシェア,https://faq.carshare.dmkt-sp.jp/faq/show/3,383,"[383, 501, 569, 325, 329]",1,"[https://faq.carshare.dmkt-sp.jp/faq/show/3, h..."
9,BMWのレンタカーとは、どんなサービスですか？,ｄカーシェア,https://faq.carshare.dmkt-sp.jp/faq/show/5,303,"[320, 286, 321, 289, 319]",NOT_MATCHED,"[https://faq.carshare.dmkt-sp.jp/faq/show/393,..."


In [17]:
new_df.to_csv('classification_tag_15.12.2020.csv')

## corpus analysis

In [21]:
# 会員登録時にsmsにて電話番号認証をするのはなぜですか
merged = pd.read_csv('/home/ifte/resource/docomo_dic/data/corpus.csv')

In [ ]:
merged[merged.page == 591].text.values

In [10]:
merged_all = pd.read_csv('/home/ifte/resource/docomo_dic/data/merged_corpus.csv')

In [ ]:
merged_all[merged_all.page == 591].text.values

In [13]:
merged_all[merged_all.page == 591].index

Int64Index([591], dtype='int64')

In [14]:
processed = '/home/ifte/resource/docomo_dic/data/processed_texts.txt'
with open(processed) as f:
    data = f.readlines()

In [20]:
for index, line in enumerate(data):
    if index == 146:
        print(line)

特典 ご 注意事項 dマーケット 特典 ご 注意事項 マスチャレ 特典 ポイント ご 注意事項 キャンペーン 内容 マスターチャレンジパスポート 獲得 dマーケット マスター お客様 本 キャンペーン 対象 サービス お 買い物  dポイント  期間 用途 限定 進呈 dマーケット マスター チャレンジャー なす 通常 d ポイント キャンペーン 対象 なる お 買い物 期間 毎月 毎月 末 キャンペーン 対象 サービス  dショッピング   dブック   dトラベル  d fashion  dミュージック   dデリバリー   dカーシェア  dtv レンタル  dアニメストア  レンタル グッズ  dエンジョイパス   dフォト   dゲーム  d ゲームプレイ チケット  dミュージック  dtv dアニメストア 月額 サービス 対象外  dポイント  期間 用途 限定 進呈 対象者 dマーケット マスター お客様 キャンペーン期間 対象 サービス 対象商品 コンテンツ ご 購入 マスターチャレンジパスポート 獲得 商品 コンテンツ ご 購入 ドコモ ご 契約 携帯電話 回線 d アカウント  dポイント  期間 用途 限定 進呈 マスターチャレンジパスポート dマーケット マスター お客様 マスターチャレンジパスポート 獲得 ダクト 本 キャンペーン 対象 サービス お 買い物 最大 倍  dポイント  期間 用途 限定 進呈 パスポート 獲得 同 月内 購入 商品 point-up 対象 なる パスポート 獲得 いただく 翌月 獲得 不要 獲得 方法 詳細 ご 確認 くださる 同 月内 商品 購入 マスターチャレンジパスポート 獲得 ポイント 進呈 対象 なる dマーケット マスター お客様 パスポート 獲得 いただける  dポイント  期間 用途 限定 進呈 進呈 期限 進呈 ご 購入 商品 コンテンツ お 支払い 確定 翌月 中旬 頃 ご 利用 期限  dポイント  期間 用途 限定 進呈 進呈 末 商品 注文 商品 発送 お 支払い 確定 当該 商品 かかる ポイント 中旬 頃 進呈 雅語 利用 期限 失効 なる 本 キャンペーン 進呈 ポイント 上限 ポイント 期間 用途 限定 なる 進呈 ポイント 数 計算 原則 お 支払い 確定 翌月 初旬 頃